In [1]:
from agential.cog.reflexion.strategies.qa import ReflexionReActQAStrategy
from agential.llm.llm import LLM
from dotenv import load_dotenv
import os

load_dotenv()

llm = LLM("gpt-3.5-turbo", organization=os.getenv("OPENAI_ORGANIZATION"))



In [ ]:
from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
from agential.cog.reflexion.prompts import HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, REFLEXION_REACT_INSTRUCTION_HOTPOTQA, REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA


question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"
strat = ReflexionReActQAStrategy(llm=llm)
out = strat.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_HOTPOTQA, 
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    patience=1,
    reset=True,
)

In [ ]:
out

In [ ]:
from agential.cog.fewshots.gsm8k import GSM8K_FEWSHOT_EXAMPLES_COT
from agential.cog.reflexion.prompts import GSM8K_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT, REFLEXION_COT_INSTRUCTION_GSM8K, REFLEXION_COT_REFLECT_INSTRUCTION_GSM8K
from agential.cog.reflexion.strategies.math import ReflexionReActMathStrategy

question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
key = -9867630

strat = ReflexionReActMathStrategy(llm=llm)
out = strat.generate(
    question=question,
    key=key,
    examples=GSM8K_FEWSHOT_EXAMPLES_COT,
    prompt=REFLEXION_COT_INSTRUCTION_GSM8K,
    reflect_examples=GSM8K_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    reflect_prompt=REFLEXION_COT_REFLECT_INSTRUCTION_GSM8K,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    patience=3,
    reset=True,
)

In [2]:
from agential.cog.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_REACT
from agential.cog.fewshots.mbpp import MBPP_FEWSHOT_EXAMPLES_REACT
from agential.cog.reflexion.prompts import HUMANEVAL_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, MBPP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, REFLEXION_REACT_INSTRUCTION_HUMANEVAL, REFLEXION_REACT_INSTRUCTION_MBPP, REFLEXION_REACT_REFLECT_INSTRUCTION_HUMANEVAL, REFLEXION_REACT_REFLECT_INSTRUCTION_MBPP
from agential.cog.reflexion.strategies.code import ReflexionReActCodeStrategy, ReflexionReActMBPPStrategy


inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
key = f"{inst['test']}\ncheck({inst['entry_point']})"

agent = ReflexionReActMBPPStrategy(
    llm=llm, 
)
out = agent.generate(
    question=question,
    key=key,
    examples=MBPP_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_MBPP,
    reflect_examples=MBPP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_MBPP,
    reflect_strategy="reflexion",
    additional_keys={"tests": key},
    reflect_additional_keys={"tests": key},
    patience=3,
    reset=True,
)

<PROMPT AGENT=============================================>
Answer a coding question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be three types:
(1) Implement[code], which implements the function to answer the question.
(2) Test[code], which implements assert statement test cases to test the implemented code.
(3) Finish[answer], which returns the code implementation and finishes the task.
You have a maximum of 6 steps.

Here are some examples:
You are an expert Python programmer, and here is your task: Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

assert set(similar_elements((3, 4, 5, 6), (5, 7, 4, 10))) == set((4, 5))
assert set(similar_elements((1, 2, 3, 4), (5, 4, 3, 7))) == set((3, 4))
assert set(similar_elements((11, 12, 14, 13), (17, 15, 14, 13))) == set((13, 14))

Thought 1: I need to write a function that accepts two t

In [3]:
out.total_tokens

28783

In [4]:
agent.llm.total_tokens

28783